### This progressive barchart implementation  (using widget.update()) freezes the browser

NB: The final result is OK (after waiting a lot) but the ticks on the y axis are not entirely displayed

In [1]:
import numpy as np
import time
import asyncio as aio
import pandas as pd
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
                  'x': {'field': 'x', 'type': 'ordinal'},
                  'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 1_000
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
display(widget)
async def progressive_loop(widget, nbins=50):
    hist = np.zeros(nbins)
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist,
        })
        widget.update('data', remove='datum.x>=0')
        widget.update('data',insert=source.to_dict(orient='records'))

await progressive_loop(widget)

VegaWidget()

### The previous implementation splitted in two cells does not freeze the browser anymore

NB: the widget blinks, the issue of ticks on the y axis persists 

#### First cell

The following cell creates the empty widget and display it:

In [ ]:
import numpy as np
import time
import pandas as pd
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
             'x': {'field': 'x', 'type': 'ordinal'},
              'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()
nsamples = 100_000
steps = 1_000
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
hist = np.zeros(nbins)
widget = VegaWidget(spec=spec_no_data)
display(widget)

#### Second cell

The following cell updates the widget in a loop:

In [ ]:
await progressive_loop(widget)

NB: I tried to introduce a delay between the widget creation and the update's loop instead of splitting the code in two cells but the problem wasn't fixed this way ...

### This implementation works fine

This implementation is almost the same but it is setting widget.spec instead of using widget.update(). It works fine: no blinks, the ticks are OK

In [ ]:
import numpy as np
import altair as alt
import pandas as pd
from vega.widget import VegaWidget
nbins = 50
spec_no_data = {
    '$schema': 'https://vega.github.io/schema/vega-lite/v4.8.1.json',
     'config': {'view': {'continuousHeight': 300, 'continuousWidth': 400}},
     'data': {'name': 'data'},
     'encoding': {
             'x': {'field': 'x', 'type': 'ordinal'},
              'y': {'field': 'y', 'type': 'quantitative'}
             },
     'mark': 'bar'}
spec_with_data = spec_no_data.copy()

nsamples = 100_000
steps = 1_000
rdata = np.random.normal(loc=0, scale=0.1, size=nsamples)
widget = VegaWidget(spec=spec_no_data)
display(widget)
async def progressive_loop2(widget, nbins):
    hist = np.zeros(nbins)
    for sl in np.split(rdata, steps):
        await aio.sleep(0.1)
        h, _ = np.histogram(sl, bins=nbins, range=(-1.0,1.0))
        hist += h
        source = pd.DataFrame({
            'x': range(nbins),
            'y': hist  # /np.sum(hist),
        })
        spec_with_data["data"] = {
            "name": "data",
            "values": source.to_dict(orient='records'),
        }
        widget.spec = spec_with_data

await progressive_loop2(widget, nbins)
